# Interpolation 

$$
\varepsilon = \int_0^\varepsilon \alpha(x - y) \varepsilon(x) dy 
$$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
rng = np.random.default_rng()
x = np.linspace(-3, 3, 5000)

z_J = np.exp(-x**2) + 0.1 * rng.standard_normal(5000)

x_J = x
x_M = x
RR = 0.2

x_M = x_J if x_M is None else x_M
alpha_r_MJ = np.exp(-((x_J[None, ...] - 
                            x_M[..., None]) ** 2 
                        / (2 * RR ** 2)))
a_M = np.trapz(alpha_r_MJ, x_J, axis=-1)
alpha_r_MJ /= a_M[..., None]
z_MJ = np.einsum('MJ,J...->MJ...', alpha_r_MJ, z_J)
z_M = np.trapz(z_MJ, x_J, axis=-1)
z_M

# Now construct the spline

In [ ]:
from scipy.interpolate import CubicSpline

spl = CubicSpline(x_M, z_M, bc_type='natural')

xs = np.linspace(-3, 3, 1000)

plt.plot(x_J, z_J, 'o', color='red', lw=4, alpha=0.8)

plt.plot(xs, spl(xs), 'b', lw=1)



In [ ]:
dz_poly = spl.derivative()
plt.plot(x_M, spl(x_M, 1))

In [ ]:
plt.plot(x_J, z_J)
plt.plot(x_M, z_M)

In [ ]:
def get_z_MN_ironed(x_J, z_J, RR, x_M=None):
    """
    Calculates the ironed z-coordinate field 
    with radial averaging.

    Args:
        x_JK: Array of x-coordinates.
        y_JK: Array of y-coordinates.
        z_JK: Array of z-coordinates.
        RR: Radial averaging parameter.

    Returns:
        Ironed z-coordinate field 
        with radial averaging.
    """
    x_M = x_J if x_M is None else x_M
    alpha_r_MJ = np.exp(-((x_J[None, ...] - 
                                x_M[..., None]) ** 2 
                            / (2 * RR ** 2)))
    a_M = np.trapz(alpha_r_MJ, x_J, axis=-1)
    alpha_r_MJ /= a_M[..., None]
    z_MJ = np.einsum('MJ,J...->MJ...', alpha_r_MJ, z_J)
    return np.trapz(z_MJ, x_J, axis=-1)


## Numpy index tricks - broadcasting and index summation rule

In [ ]:
import numpy as np
a = np.linspace(0,10,11)
a

In [ ]:
sig = np.linspace(0,3,9).reshape(3,3)
eps = np.linspace(3,6,9).reshape(3,3)
sig

In [ ]:
np.einsum('ij,kj->ik', sig, eps)

In [ ]:
DELTA = np.identity(3)

EPS = np.zeros((3, 3, 3), dtype='f')
EPS[(0, 1, 2), (1, 2, 0), (2, 0, 1)] = 1
EPS[(2, 1, 0), (1, 0, 2), (0, 2, 1)] = -1

In [ ]:
EPS # Levi-Civita symbol
EPS

In [ ]:
u = np.array([1,2,3])
v = np.array([4,3,-3])

In [ ]:
np.einsum('ijk,...j,...k->...i',EPS,u,v)